# NBA MCP Synthesis Workflow

This notebook demonstrates the complete multi-model synthesis workflow with MCP context.

## What This Demonstrates
1. Connecting to MCP server
2. Gathering context for different query types
3. Running multi-model synthesis (DeepSeek + Claude + Ollama)
4. Analyzing results and costs

In [ ]:
# Setup
import sys
import os
import asyncio
import json
from pathlib import Path
from datetime import datetime

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Load environment
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

# Import synthesis function
from synthesis.multi_model_synthesis import synthesize_with_mcp_context

print("✓ Environment loaded")

## Example 1: SQL Query Optimization

In [ ]:
# Example SQL query that needs optimization
slow_query = """
SELECT *
FROM player_game_stats
WHERE player_id IN (
    SELECT player_id FROM players WHERE team_id = 'LAL'
)
ORDER BY game_date DESC
"""

result = await synthesize_with_mcp_context(
    user_input="Optimize this SQL query for better performance",
    selected_code=slow_query,
    query_type="sql_optimization",
    enable_ollama_verification=False,  # Set to True if Ollama is running
    output_dir=str(project_root / "synthesis_output")
)

print("\n" + "="*80)
print("SYNTHESIS RESULTS")
print("="*80)
print(f"Status: {result['status']}")
print(f"Models used: {', '.join(result['models_used'])}")
print(f"Total tokens: {result['total_tokens']}")
print(f"Total cost: ${result['total_cost']:.4f}")
print(f"Execution time: {result['execution_time_seconds']:.2f}s")
print(f"\nFinal explanation:\n{result['final_explanation'][:500]}...")

## Example 2: Statistical Analysis

In [ ]:
result2 = await synthesize_with_mcp_context(
    user_input="Analyze the correlation between three-point shooting percentage and wins for the 2023-24 season",
    query_type="statistical_analysis",
    enable_ollama_verification=False,
    output_dir=str(project_root / "synthesis_output")
)

print(f"\nStatus: {result2['status']}")
print(f"Cost: ${result2['total_cost']:.4f}")
print(f"Time: {result2['execution_time_seconds']:.2f}s")

## Example 3: ETL Code Generation

In [ ]:
result3 = await synthesize_with_mcp_context(
    user_input="Generate an ETL pipeline to load box score data from S3 into the games table",
    query_type="etl_generation",
    enable_ollama_verification=False,
    output_dir=str(project_root / "synthesis_output")
)

print(f"\nStatus: {result3['status']}")
print(f"Cost: ${result3['total_cost']:.4f}")
if result3.get('final_code'):
    print(f"\nGenerated code ({len(result3['final_code'])} characters)")
    print(result3['final_code'][:300] + "...")

## Cost Analysis

In [ ]:
# Analyze total costs
total_cost = sum([
    result.get('total_cost', 0),
    result2.get('total_cost', 0),
    result3.get('total_cost', 0)
])

total_tokens = sum([
    result.get('total_tokens', 0),
    result2.get('total_tokens', 0),
    result3.get('total_tokens', 0)
])

print("\n" + "="*80)
print("COST SUMMARY")
print("="*80)
print(f"Total queries: 3")
print(f"Total tokens: {total_tokens:,}")
print(f"Total cost: ${total_cost:.4f}")
print(f"Average cost per query: ${total_cost/3:.4f}")
print("\nCost breakdown:")
print(f"  Query 1 (SQL optimization): ${result.get('total_cost', 0):.4f}")
print(f"  Query 2 (Statistical analysis): ${result2.get('total_cost', 0):.4f}")
print(f"  Query 3 (ETL generation): ${result3.get('total_cost', 0):.4f}")